In [ ]:
import pandas as pd
import numpy as np
from numpy import NaN
from itertools import chain
import json


In [ ]:
# напрямую с файлами
ac = pd.read_csv('results/1С-аналитик_vac.csv', sep=';', encoding='utf-8')
ds = pd.read_csv('results/Data Scientist_vac.csv', sep=';', encoding='utf-8')
do = pd.read_csv('results/DataOps-инженер_vac.csv', sep=';', encoding='utf-8')
ab = pd.read_csv('results/Аналитик BI_vac.csv', sep=';', encoding='utf-8')
ad = pd.read_csv('results/Аналитик данных_vac.csv', sep=';', encoding='utf-8')
# dj = pd.read_csv('results/Дата-журналист_vac.csv', sep=';', encoding='utf-8')
de = pd.read_csv('results/Дата-инженер_vac.csv', sep=';', encoding='utf-8')
ma = pd.read_csv('results/Маркетинговый аналитик_vac.csv', sep=';', encoding='utf-8')
pa = pd.read_csv('results/Продуктовый аналитик_vac.csv', sep=';', encoding='utf-8')
sa = pd.read_csv('results/Системный аналитик_vac.csv', sep=';', encoding='utf-8')
fa = pd.read_csv('results/Финансовый аналитик_vac.csv', sep=';', encoding='utf-8')
ba = pd.read_csv('results/Бизнес-аналитик_vac.csv', sep=';', encoding='utf-8')


In [ ]:
# добавление меток широта-долгота
with open('geo_rus.json', encoding='utf-8') as geo:
    cities = json.load(geo)

def lat(row):
    for el in cities:
        if row == el['city']:
            return float(el['lat'])
        
def lon(row):
    for el in cities:
        if row == el['city']:
            return float(el['lon'])


In [ ]:
full = pd.concat([ds, do, ab, ad, dj, de, ma, sa, ac, fa, ba, pa])
# full.to_csv('vacancies.csv', index=False)

full['lat'] = full['city'].apply(lat)
full['lon'] = full['city'].apply(lon)
full = full[['id', 'spec', 'job_title', 'city', 'lat', 'lon', 
         'employer', 'schedule', 'skills', 'experience', 
         'published', 'salary_from', 'salary_to', 'currency']]

In [ ]:
ac_sk = ['1c|1с', 'agile|scrum|kanban', 'управление', 'erp', 'отчет',
         'анализ', 'тз', 'srs', 'моделирование', 'bpmn', 'тестирование',
         'документ']

ds_sk = ['анализ', 'sheets', 'статистика', 'data studio', 'python', 'sql',
         'ml', 'machine learning', 'сбор', 'визуализация', 'исследование',
         'нейронные сети', 'нейросети', 'временные ряды', 'алгоритм',
         'sklearn', 'tensorflow', 'pytorch','cv', 'gan', 'nlp']

de_do_sk = ['sql', 'базы данных', 'etl', 'power bi|powerbi', 'python', 
            'pandas', 'numpy', 'hadoop', 'mapreduce', 'apache', 'kafka', 
            'spark', 'airflow', 'clickhouse', 'yandex cloud', 'docker', 
            'kubernetes', 'ansible', 'gitlab ci', 'mlflow']

ab_sk = ['анализ', 'sheets', 'excel', 'статистика', 'визуализация', 
         'data studio', 'power bi|powerbi', 'pivot', 'datalens', 'tableau',
         'сбор', 'sql', 'ssas']

ad_sk = ['анализ', 'sheets', 'excel', 'статистика', 'визуализация',
         'data studio', 'python', 'jupyter notebook', 'numpy', 
         'pandas', 'сбор', 'sql', 'ml', 'machine learning', 'hadoop']

dj_sk = ['анализ', 'сбор', 'очистка', 'визуализация', 'построение карт', 'сторителлинг',
         'python', 'pandas', 'статистика', 'tableau']

ma_sk = ['анализ', 'sheets', 'статистика', 'визуализация', 'google looker studio',
         'метрики', 'юнит-экономика', 'поиск точек роста', 'RFM-анализ', 
         'когортный анализ', 'веб-аналитика', 'яндекс метрика', 'google analitics',
         'ab', 'sql', 'power bi|powerbi', 'python', 'numpy', 'pandas', 'api']

pa_sk = ['анализ', 'юнит-экономика', 'ab', 'sql', 'tableau', 'power bi|powerbi', 
         'python', 'numpy', 'pandas', 'исследов']

sa_sk = ['разработка по', 'agile', 'scrum', 'моделирование бизнес-процессов',
         'нотации', 'проектирование', 'сбор', 'описание', 'визуализация',
         'прототипирование', 'figma', 'sql', 'rest api', 'soap', 'open api',
         'проектная документация', 'cjm', 'тз', 'спецификация', 'SRS', 'bcm']

fa_sk = ['1с', 'бухгалтер', 'финансовая', 'анализ', 'управление', 'статистика', 
         'план-фактный анализ', 'планирование', 'моделирование', 'презентац',
         'powerpoint', 'excel', 'визуализация']

ba_sk = ['моделирование', 'анализ', 'оптимизация', 'отчетност', 'сбор',
         'tableau', 'bi', 'исследовани', 'agile|scrum|kanban', 'тз', 'документ']


#### 1 / 12 Аналитик данных, подготовка к конкату

In [ ]:
ad.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ad.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ad.skills.str.split(',').map(len)

ad = pd.DataFrame({'id': np.repeat(ad['id'], lens),
                    'spec': np.repeat(ad['spec'], lens),
                    'skills': chainer(ad['skills'])})
# ad.head()
lens

In [ ]:
ad = ad.loc[ad.skills.str.contains('|'.join(ad_sk))]
ad.skills = ad.skills.str.strip()

ad.skills.loc[(ad.skills.str.contains('sql'))] = 'sql'
ad.skills.loc[(ad.skills.str.contains('excel'))] = 'excel'
ad.skills.loc[(ad.skills.str.contains('machine learning'))] = 'ml'
ad.skills.loc[(ad.skills.str.contains('анализ'))] = 'анализ (данных, финансовый, т.д.)'

#### 2 / 12 Data Scientist, подготовка к конкату

In [ ]:
ds.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ds.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ds.skills.str.split(',').map(len)

ds = pd.DataFrame({'id': np.repeat(ds['id'], lens),
                    'spec': np.repeat(ds['spec'], lens),
                    'skills': chainer(ds['skills'])})

In [ ]:
ds = ds.loc[ds.skills.str.contains('|'.join(ds_sk))]
ds.skills = ds.skills.str.strip()

ds.skills.loc[(ds.skills.str.contains('sql'))] = 'sql'
ds.skills.loc[(ds.skills.str.contains('excel'))] = 'excel'
ds.skills.loc[(ds.skills.str.contains('machine learning'))] = 'ml'
ds.skills.loc[(ds.skills.str.contains('ml'))] = 'ml'
ds.skills.loc[(ds.skills.str.contains('анализ'))] = 'анализ (данных, финансовый, т.д.)'

#### 3 / 12 Аналитик BI, подготовка к конкату

In [ ]:
ab.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ab.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ab.skills.str.split(',').map(len)

ab = pd.DataFrame({'id': np.repeat(ab['id'], lens),
                    'spec': np.repeat(ab['spec'], lens),
                    'skills': chainer(ab['skills'])})

In [ ]:
ab = ab.loc[ab.skills.str.contains('|'.join(ab_sk))]
ab.skills = ab.skills.str.strip()

ab.skills.loc[(ab.skills.str.contains('sql'))] = 'sql'
ab.skills.loc[(ab.skills.str.contains('excel'))] = 'excel'
ab.skills.loc[(ab.skills.str.contains('bi'))] = 'power bi'
ab.skills.loc[(ab.skills.str.contains('pivot'))] = 'pivot'
ab.skills.loc[(ab.skills.str.contains('анализ'))] = 'анализ (данных, финансовый, т.д.)'

#### 4 / 12 Системный аналитик, подготовка к конкату

In [ ]:
sa.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
sa.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = sa.skills.str.split(',').map(len)

sa = pd.DataFrame({'id': np.repeat(sa['id'], lens),
                    'spec': np.repeat(sa['spec'], lens),
                    'skills': chainer(sa['skills'])})

In [ ]:
sa = sa.loc[sa.skills.str.contains('|'.join(sa_sk))]
sa.skills = sa.skills.str.strip()

sa.skills.loc[(sa.skills.str.contains('sql'))] = 'sql'
sa.skills.loc[(sa.skills.str.contains('agile'))] = 'agile'
sa.skills.loc[(sa.skills.str.contains('тз'))] = 'написание тз'
sa.skills.loc[(sa.skills.str.contains('soap'))] = 'soap'
sa.skills.loc[(sa.skills.str.contains('моделирование | проектирование'))] = 'проектирование (интерфейсов, api, т.д.)'
sa.skills.loc[(sa.skills.str.contains('сбор требований'))] = 'сбор требований'

#### 5 / 12 Маркетинговый аналитик, подготовка к конкату

In [ ]:
ma.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ma.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ma.skills.str.split(',').map(len)

ma = pd.DataFrame({'id': np.repeat(ma['id'], lens),
                    'spec': np.repeat(ma['spec'], lens),
                    'skills': chainer(ma['skills'])})

In [ ]:
ma = ma.loc[ma.skills.str.contains('|'.join(ma_sk))]
ma.skills = ma.skills.str.strip()

ma.skills.loc[(ma.skills.str.contains('анализ'))] = 'анализ (данных, финансовый, т.д.)'
ma.skills.loc[(ma.skills.str.contains('sql'))] = 'sql'
ma.skills.loc[(ma.skills.str.contains('статистика'))] = 'статистика'

#### 6 / 12 Дата-журналист, подготовка к конкату

In [ ]:
dj.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
dj.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = dj.skills.str.split(',').map(len)

dj = pd.DataFrame({'id': np.repeat(dj['id'], lens),
                    'spec': np.repeat(dj['spec'], lens),
                    'skills': chainer(dj['skills'])})

In [ ]:
dj = dj.loc[dj.skills.str.contains('|'.join(dj_sk))]
dj.skills = dj.skills.str.strip()

dj.skills.loc[(dj.skills.str.contains('анализ'))] = 'анализ (данных, финансовый, т.д.)'
dj.skills.loc[(dj.skills.str.contains('сбор'))] = 'сбор данных'
dj.skills.loc[(dj.skills.str.contains('визуализация'))] = 'визуализация данных'
dj.skills.loc[(dj.skills.str.contains('storytelling'))] = 'сторителлинг'
dj.skills.loc[(dj.skills.str.contains('story telling'))] = 'сторителлинг'

#### 7-8 / 12 Дата-инженер | DataOps-инженер, подготовка к конкату

In [ ]:
#DataOps
do.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
do.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = do.skills.str.split(',').map(len)

do = pd.DataFrame({'id': np.repeat(do['id'], lens),
                    'spec': np.repeat(do['spec'], lens),
                    'skills': chainer(do['skills'])})

In [ ]:
do = do.loc[do.skills.str.contains('|'.join(de_do_sk))]
do.skills = do.skills.str.strip()

do.skills.unique()

# do.skills.loc[(do.skills.str.contains('...'))] = '...'
# do.skills.loc[(do.skills.str.contains('...'))] = '...'

In [ ]:
# дата-инженер
de.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
de.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = de.skills.str.split(',').map(len)

de = pd.DataFrame({'id': np.repeat(de['id'], lens),
                    'spec': np.repeat(de['spec'], lens),
                    'skills': chainer(de['skills'])})

In [ ]:
de = de.loc[de.skills.str.contains('|'.join(de_do_sk))]
de.skills = de.skills.str.strip()

de.skills.loc[(de.skills.str.contains('sql'))] = 'sql'
de.skills.loc[(de.skills.str.contains('bi'))] = 'power bi'
de.skills.loc[(de.skills.str.contains('airflow'))] = 'airflow'
de.skills.loc[(de.skills.str.contains('nifi|ni-fi'))] = 'nifi'

#### 9 / 12 Финансовый аналитик, подготовка к конкату

In [ ]:
fa.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
fa.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = fa.skills.str.split(',').map(len)

fa = pd.DataFrame({'id': np.repeat(fa['id'], lens),
                    'spec': np.repeat(fa['spec'], lens),
                    'skills': chainer(fa['skills'])})

In [ ]:
fa = fa.loc[fa.skills.str.contains('|'.join(fa_sk))]
fa.skills = fa.skills.str.strip()

fa.skills.loc[(fa.skills.str.contains('1с|1c'))] = '1с (предприятие, бухгалтерия)'
fa.skills.loc[(fa.skills.str.contains('excel'))] = 'excel'
fa.skills.loc[(fa.skills.str.contains('powerpoint|презент'))] = 'powerpoint'
fa.skills.loc[(fa.skills.str.contains('анали'))] = 'анализ (данных, финансовый, т.д.)'
fa.skills.loc[(fa.skills.str.contains('управление'))] = 'управление (рисками, затратами, т.д.)'
fa.skills.loc[(fa.skills.str.contains('планирование'))] = 'планирование (финансовое, инвестиционное, т.д.)'
fa.skills.loc[(fa.skills.str.contains('моделирование'))] = 'моделирование (финансовое, экономическое, т.д.)'
fa.skills.loc[(fa.skills.str.contains('бухгал'))] = 'бухучёт'
fa.skills.loc[(fa.skills.str.contains('статистика'))] = 'статистика'
fa.skills.loc[(fa.skills.str.contains('визуализация'))] = 'визуализация'

#### 10 / 12 1С-аналитик, подготовка к конкату

In [ ]:
ac.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ac.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ac.skills.str.split(',').map(len)

ac = pd.DataFrame({'id': np.repeat(ac['id'], lens),
                    'spec': np.repeat(ac['spec'], lens),
                    'skills': chainer(ac['skills'])})

In [ ]:
ac = ac.loc[ac.skills.str.contains('|'.join(ac_sk))]
ac.skills = ac.skills.str.strip()

ac.skills.loc[(ac.skills.str.contains('1с|1c'))] = '1с (предприятие, бухгалтерия)'
ac.skills.loc[(ac.skills.str.contains('отчетность'))] = 'отчетность (налоговая, бухгалтерская)'
ac.skills.loc[(ac.skills.str.contains('powerpoint'))] = 'powerpoint'
ac.skills.loc[(ac.skills.str.contains('документ'))] = 'проектная документация'

#### 11 / 12 Продуктовый аналитик, подготовка к конкату

In [ ]:
pa.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
pa.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = pa.skills.str.split(',').map(len)

pa = pd.DataFrame({'id': np.repeat(pa['id'], lens),
                    'spec': np.repeat(pa['spec'], lens),
                    'skills': chainer(pa['skills'])})

In [ ]:
pa = pa.loc[pa.skills.str.contains('|'.join(pa_sk))]
pa.skills = pa.skills.str.strip()

pa.skills.loc[(pa.skills.str.contains('sql'))] = 'sql'
pa.skills.loc[(pa.skills.str.contains('анализ|analys'))] = 'анализ (данных, финансовый, т.д.)'
pa.skills.loc[(pa.skills.str.contains('bi'))] = 'power bi'
pa.skills.loc[(pa.skills.str.contains('исследов'))] = 'проведение исследований'

#### 12 / 12 Бизнес-аналитик, подготовка к конкату

In [ ]:
ba.drop(['job_title', 'city', 'employer', 'schedule', 'experience',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ba.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ba.skills.str.split(',').map(len)

ba = pd.DataFrame({'id': np.repeat(ba['id'], lens),
                    'spec': np.repeat(ba['spec'], lens),
                    'skills': chainer(ba['skills'])})

In [ ]:
ba = ba.loc[ba.skills.str.contains('|'.join(ba_sk))]
ba.skills = ba.skills.str.strip()

ba.skills.loc[(ba.skills.str.contains('анализ'))] = 'анализ (данных, финансовый, т.д.)'
ba.skills.loc[(ba.skills.str.contains('моделирование'))] = 'моделирование (финансовое, экономическое, т.д.)'
ba.skills.loc[(ba.skills.str.contains('отчет'))] = 'отчетность (налоговая, бухгалтерская)'
ba.skills.loc[(ba.skills.str.contains('bi'))] = 'bi-системы'
ba.skills.loc[(ba.skills.str.contains('документ'))] = 'проектная документация'
ba.skills.loc[(ba.skills.str.contains('оптимизация'))] = 'оптимизация процессов'
ba.skills.loc[(ba.skills.str.contains('исследов'))] = 'проведение исследований'
ba.skills.loc[(ba.skills.str.contains('тз'))] = 'составление тз'
ba.skills.loc[(ba.skills.str.contains('сбор'))] = 'сбор требований'
ba.skills.loc[(ba.skills.str.contains('agile'))] = 'agile'

#### Конкат + топ-15 навыков + сводная

In [ ]:
df = pd.concat([ds, do, ab, ad, dj, de, ma, sa, ac, fa, ba, pa])
df.skills.replace(NaN, '-', inplace=True)
df

In [ ]:
top_15 = df.skills.value_counts().to_frame().reset_index().head(15)
list_top = list(top_15.skills)
list_top

In [ ]:
df = df[df.skills.isin(list_top)]

In [ ]:
df = pd.pivot_table(
    df,
    index='skills',
    columns='spec',
    values='id',
    aggfunc='count'
).reset_index()

In [ ]:
df

In [ ]:
df.to_excel('key_skills.xlsx', index=False)

# Карта РФ

In [ ]:
df = full.groupby('city')['id'].count()
df

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(full, lat="lat", lon="lon",
                        size='id', size_max=5,)
fig.update_layout(
        mapbox=dict(
            accesstoken='pk.eyJ1IjoiZHVtYmZpc2giLCJhIjoiY2xobTc3YXVrMTltZzNocXAwNzQxcG9tdSJ9.HyrRTRTCyDWgZdaFWadHxQ',
            style='mapbox://styles/dumbfish/clhm8mm6c01n701pr051pebv8'
        )
    )